In [3]:
import requests
import pyarabic.araby as araby # for processing arabic 
import re

In [4]:
#reading the CAMEO arabic rules file
rules =[]
with open('CAMEO.ar.0.2.txt', 'rb') as input_file :
    for line in input_file : 
        if line != '':
            rules.append(line.decode('utf-8') )
        
rules = [i.replace('\n','') for i in rules]
# rules = [i for i in rules if i !='']
rules[:30]

['# LAST UPDATE: 11 July 2018',
 '',
 '####### SYNONYM SETS ####### ',
 '&CURRENCY',
 '',
 '+درهم_مغربي_',
 '+فرنك_كونغولي_',
 '+ناكفا_',
 '+فرنك_سويسري_',
 '+نيرة_نيجيرية_',
 '+تالا_',
 '+روبية_نيبالية_',
 '+يورو_',
 '+بيسو_فلبيني_',
 '+كوردبا_نيكاراغوا_',
 '+كرون_إستوني_',
 '+ساماني_طاجيكي_',
 '+دينار_تونسي_',
 '+دولار_غياني_',
 '+سيدي_',
 '+مارك_',
 '+نغولترم_',
 '+دولار_جامايكي_',
 '+بات_',
 '+هريفنا_',
 '+دولار_البليز_',
 '+بيزو_أرجنتيني_',
 '+أوقية_',
 '+دينار_',
 '+درام_أرميني_']

In [5]:
x = ''
for i in rules :
    if i.startswith('- *') : 
        x = i
        break
y = x.split('#')
y[0]

'- * (ل حد)   [154] '

## UDpipe approach

In [9]:
# http://lindat.mff.cuni.cz/services/udpipe/api/process?tokenizer&tagger&model=arabic-ud-2.0-170801&data=الجرائم
acceptable_formats = ['-*[]',"-*()[]", "-*{}[]"  , "-*({})[]", '-*()[]' , '-*()()[]', '-*({})()[]', '-*{()}[]' , '-*(())[]', '-*({()})[]' , '-*(){}[]' , '-*{}()[]' , '-*(){}[]']
special_formats = ['-*()', '-*()()[]' , '', '-*(}})[]' , '-*}}[]', '-*{}[]' , '-*([]' , '-*({({()[]'  , '-*(}()(}[]' , '-*))[]', '-*((}(})[]']
wrong_skip = ['* نزع سلاح) جهود)' , ' * عن تقرير مصير) بيان)', '* عن تقرير مصير) بيان)' , '* ((مساعدات انسانية} (من أجل})']
encod_chars = ['-' , '*' , '(' , ')' , '[' , ']' , '{','}']
#store things for later
special = []
to_be_replaced = []
error =[]

def check_and_reconstruct_rule(rule): 
    rule = ' '.join(rule.split()) # to remove all white spaces and duplicate ones 
    if rule == '' :
        return 'ERROR'
    
    rule_encod = ''.join([i for i in rule if i in encod_chars])
#     print('RULE : {} || ENCODE : {}'.format(rule , rule_encod))
    
    if rule_encod in acceptable_formats: #verb is already correct(rare cases)- apply regex stright
        return rule
    elif rule_encod in special_formats:
#         print("YES")
        return 'SPECIAL'
    else : 
        if rule_encod == '-*({{)[]': # -*({{)[]
            t = rule.find('{')
            rule = rule[:t] + rule[t+1: -1] + '})'
            rule_encod =''.join([i for i in rule if i in encod_chars])
        elif rule_encod  =='*((':
            t =  rule.find('(')
            rule = rule[:t] + rule[t+1:] + ')'
            rule_encod =''.join([i for i in rule if i in encod_chars])
        elif rule_encod  =='*{{':
            t =  rule.find('{')
            rule = rule[:t] + rule[t+1:] + '}'
            rule_encod =''.join([i for i in rule if i in encod_chars])
        elif rule_encod =='*{((}' : 
            t = rule.find('(')
            rule = rule[:t] + rule[t+1: -1] + ')}'
            rule_encod =''.join([i for i in rule if i in encod_chars])
        elif rule_encod == '*{){)': 
            t = rule.find('{')
            rule = rule[:t] + '(' + rule[t+1: -1] + '}'
            rule_encod =''.join([i for i in rule if i in encod_chars])
        elif rule_encod in['-***[]','-***()[]' , '-*()**[]'] : 
            rule = rule [3:]
            t = rule.find('*')
            rule = rule[:t] + '{' + rule[t+1:]
            t = rule.find('*')
            rule = rule[:t] + '}' + rule[t+1:]
            rule = '- *' + rule
            rule_encod =''.join([i for i in rule if i in encod_chars])
            
        
        if rule_encod not in acceptable_formats :
            print('ERR | {} | {}'.format(rule, rule_encod))
            return 'ERROR'
        else :
#             print('FIX | {} | {}'.format(rule, rule_encod))
            return rule
    
    return rule

def rule(i):
    if i.strip()[0] != '-' : # Wrong delete 
        print('NEED MANUAL FIX :{}'.format(i)) # delete those need to mananually fix
        special.append(i)
        return i
    elif re.search('[a-zA-Z]', i) : 
        to_be_replaced.append(i)
        return i
    
    elif i in wrong_skip : 
        print('SKP | {}'.format(i))
        return i
    else : 
        if '$' in i or '+' in i or '^' in i or '/' in i or '|' in i :
            i = ''.join([c for c in i if c not in set(['$', '+', '^', '/' , '|'])])
            print("EHH :{}".format(i))
        rule_fixed = check_and_reconstruct_rule(i.strip())
        if rule_fixed == i :
            return i
        elif rule_fixed == 'SPECIAL':
            print('SPECIAL :{}'.format(i))
            special.append(i)
            return 'SPECIAL'
        elif rule_fixed == 'ERROR' :
            print('RET ERR :{}'.format(i))
            error.append(i)
            return 'ERROR'
        else :  # fixed by now
#             print('is all good')
            return rule_fixed
            
        

In [49]:
for i in range(len(rules)) : 
    if rules[i].startswith('- *'): 
        sp_re = rules[i].split('#')
        o_r = sp_re[0]
        utl_part = sp_re[1]
        r = rule(o_r)
        if r =='ERROR':
            print('-----------------')
            print('old_ :{}'.format(o_r))
            print('new_ :{}'.format(r))
            print('-----------------')
            break
        elif r == 'SPECIAL':
            print('-------SPECAIL----------')
            print('old_ :{}'.format(o_r))
            print('new_ :{}'.format(r))
            print('-----------------')
        else :
            rules[i] = r + ' #{}'.format(utl_part)
#             print(rules[i])
            
        

NEED MANUAL FIX :- * (نقل جوي} (من) (إلى}   [073] 
-------SPECAIL----------
old_ :- * (نقل جوي} (من) (إلى}   [073] 
new_ :SPECIAL
-----------------
NEED MANUAL FIX :- * (نقل جوي} (من) (إلى}   [073] 
-------SPECAIL----------
old_ :- * (نقل جوي} (من) (إلى}   [073] 
new_ :SPECIAL
-----------------
NEED MANUAL FIX :- * نزع سلاح) جهد)   [105] 
-------SPECAIL----------
old_ :- * نزع سلاح) جهد)   [105] 
new_ :SPECIAL
-----------------
NEED MANUAL FIX :- * نزع سلاح) جهد)   [105] 
-------SPECAIL----------
old_ :- * نزع سلاح) جهد)   [105] 
new_ :SPECIAL
-----------------
NEED MANUAL FIX :- * عن تقرير مصير) بيان)   [1044] 
-------SPECAIL----------
old_ :- * عن تقرير مصير) بيان)   [1044] 
new_ :SPECIAL
-----------------
NEED MANUAL FIX :- * ((مساعدة إنساني} (من أجل})   [073] 
-------SPECAIL----------
old_ :- * ((مساعدة إنساني} (من أجل})   [073] 
new_ :SPECIAL
-----------------
NEED MANUAL FIX :- * ((مساعدة إنساني} (من أجل})   [073] 
-------SPECAIL----------
old_ :- * ((مساعدة إنساني} (من أجل})   [

In [181]:
with open('CAMERO.ar.txt' , 'w') as output_file:
    for line in rules: 
        output_file.write('{}\n'.format(line))
    

In [10]:
def udpipe(string) :
    # Prepping String 
    words  = re.findall(u'[\u0600-\u06FF]+', string) #getting arabic characters 
    data = ' '.join(words)
    
    pipe_base_url = 'http://lindat.mff.cuni.cz/services/udpipe/api/process?tokenizer&tagger' 
    attributes = {} 
    attributes['model'] = 'arabic-ud-2.0-170801'
    attributes['data'] = data

    data = requests.get(pipe_base_url , attributes)
    result = data.json()['result'].split('\n')
    udpipe_results = [re.findall(u'[\u0600-\u06FF]+', i) for i in result] # cleaning 
    udpipe_results = [i for i in udpipe_results if i !=[]]
    
    return udpipe_results

def udpipe_reconstruct(original_text) : 
    udpipe_results = udpipe(original_text)
    for i in range(1, len(udpipe_results)):
        if len(udpipe_results[i]) == 1 : # composite verbs get the next two words
            original_text =original_text.replace(udpipe_results[i][0] , '{} {}'.format(udpipe_results[i+1][1],udpipe_results[i+2][1]))
        else :
            original_text =original_text.replace(udpipe_results[i][0] , udpipe_results[i][1])
    original_text = araby.strip_tashkeel(original_text)
    return original_text


def master_reconstruct_input(text_input , input_type): 
    # type 0: none-rule just do just do udpipe_reconstruction
    # type 1 : fix the rule + do udpipe_reconstruction
    if input_type == 0:
        return udpipe_reconstruct(text_input)
    else : 
        return udpipe_reconstruct(rule(text_input))

## the extra verb stuff

In [7]:
new_dict =[]
with open('extra_verb_dict.txt') as inputf : 
    for i in inputf : 
        new_dict.append(i.replace('\n' , ''))

print(new_dict[:10])
print(len(new_dict))
# new_dict = [i.strip() for i in new_dict if i]
print(new_dict[:10])
print(len(new_dict))

['--- عصم  [140] ---', '', '--- فجر  [---] ---', '- *  قنابل\t\t[183]', '', '--- غول  [---] ---', '- *  فلسطيني\t\t[186]', '', '--- هدف  [---] ---', '- * * برجي مركز التجارة العالمي*\t\t[1383]']
3981
['--- عصم  [140] ---', '', '--- فجر  [---] ---', '- *  قنابل\t\t[183]', '', '--- غول  [---] ---', '- *  فلسطيني\t\t[186]', '', '--- هدف  [---] ---', '- * * برجي مركز التجارة العالمي*\t\t[1383]']
3981


ERR | --- غول [---] --- | ---[---]---
RET ERR :--- غول  [---] ---


'ERROR'

In [23]:
RULE_FORMAT_ERROR = [] 
FIXED_AND_LIMMATIZED = [] 

for i in new_dict :
    if i == '' : 
        FIXED_AND_LIMMATIZED.append('')
        continue
    if i[:3] == '---' : 
        reconstructed = master_reconstruct_input(i, 0)
    elif i[:3] == '- *': 
        reconstructed = master_reconstruct_input(i, 1)
    else : 
        print('unknown input : -{}-'.format(i))
#         print('type {}'.format(type(i)))
#     reconstructed = master_reconstruct_input(i,0 )
    if reconstructed =='ERROR': 
        break
    if reconstructed =='SPECIAL' :
        print('SPECIAL : {}'.format(i))
    else : 
        FIXED_AND_LIMMATIZED.append(reconstructed)

In [24]:
len(FIXED_AND_LIMMATIZED)

3981

In [60]:
x = '- * *الطائرات العسكرية* (لقصف) [195]'
x = x[3:]
t = x.find('*')
x = x[:t] + '{' + x[t+1:]
t = x.find('*')
x = x[:t] + '}' + x[t+1:]
x = '- *' + x 
x

'- * {الطائرات العسكرية} (لقصف) [195]'

In [72]:
for i in new_dict:
    if i[:3] != '---':
        rule(i)

In [25]:
with open('extra_verb_dict_fixed', 'w') as outputf : 
    for i in FIXED_AND_LIMMATIZED :
        outputf.write('{}\n'.format(i))
print('done !')

done !
